In [1]:
# AYU CODE
from scipy import fft
from qtn.bimax import BiMax
from qtn.util import (boltzmann, emass, echarge, permittivity, cspeed, fperp, fp)
import numpy as np
import mpmath as mp
import matplotlib.pyplot as plt
from pyspedas import wind
from pytplot import tplot, get_data
import time
%matplotlib inline

In [2]:
ant_len = 50      # m (monopole) 
ant_rad = 1.9e-4  # m
base_cap = 20e-12 # Farad
fbins = np.array([4000*1.0445**i for i in range(96)])

In [3]:
vsw=3.966e4
tc = 9.91 * echarge/boltzmann
t = 6.57
n = 0.026
tep = 9.91/9.62
tp = .3
fpe = 31.47e3
fpe = np.random.uniform(20, 70, 1000)

In [4]:
ne = 12.28e6
nc = ne/(1+n)
ldc = np.sqrt(permittivity * boltzmann * tc/ nc / echarge**2)
lrel = ant_len/ldc

In [5]:
p = BiMax(ant_len, ant_rad, base_cap)

In [6]:
wrelplots = [fbins/fpe_val for fpe_val in fpe]

In [ ]:
g_s = []
for wrelplot in wrelplots:
    g_s.append(np.array([p.gamma_shot(wrel, lrel, n, t, tc) for wrel in wrelplot]))

In [ ]:
def helper(wrel):
    """
    add precision control to bimax function.
    
    """
    if wrel > 1 and wrel < 1.2:
        mp.mp.dps = 40
    else:
        mp.mp.dps = 20
    return p.bimax(wrel, lrel, n, t, tc)

In [ ]:
enoise = []
for wrelplot in wrelplots:
    enoise.append(np.array([helper(wrel) for wrel in wrelplot]))

In [ ]:
#Modified by AYU 
# - Was running into an parameter error 
# - need to fix to match up with reference graph
pnoise = []
for wrelplot in wrelplots:
    pnoise.append(np.array([p.proton(wrel*np.sqrt(1+n), ne, n, t, tep, tc, vsw) for wrel in wrelplot]))

In [ ]:
gamma = g_s[:, :, 0]
shot = g_s[:,:, 1]
vr2 = [enoise + pnoise + shot for i in range(len(shot))]
vr2 = [[float(num) for num in vr] for vr in vr2]
vr2_n = [list(map(lambda x: np.random.normal(x, np.mean(vr2)/7), vr)) for vr in vr2]

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig = plt.figure(figsize=[6, 6])
plt.plot(fbins/1000, vr2[0]/gamma, 'o-', markersize=2, label='total')
plt.plot(fbins/1000, vr2_n[0]/gamma, 'o-', markersize=2, label='total')

#plt.plot(fbins/1000, enoise/gamma, label='electron')
#plt.plot(fbins/1000, pnoise/gamma, label='proton')
#plt.plot(fbins/1000, shot/gamma, label='shot')
plt.xscale('log')
plt.yscale('log')
#plt.xlim([4, 256])
#plt.ylim([1e-16, 5e-13])
plt.xlabel(r'$f[kHz]$')
plt.ylabel(r'$V_r^2[V^2Hz^{-1}]$')
plt.title('June 25 1995:06:33:06.432')
plt.legend(loc='best')
plt.show()

In [ ]:
import shelve 
with shelve.open("data/thetoretical_data_test") as file:
    file["spectra"] = [vr/gamma for vr in vr2]
    file["freq"] = fbins/1000
    file["fpe"] = fpe

In [ ]:
# Import all
import pandas as pd 
from sklearn.svm import OneClassSVM
import numpy as np
from numpy import where
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.DataFrame(vr2/gamma)
model = OneClassSVM(kernel = 'sigmoid', gamma = .001, nu = .5).fit(df)
y_pred = model.predict(df)
outliers = where(y_pred == -1)
print(outliers)
len(vr2)

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig = plt.figure(figsize=[6, 6])
plt.plot(fbins/1000, vr2/gamma, 'o-', markersize=2, label='total')
plt.plot(fbins/1000, enoise/gamma, label='electron')
plt.plot(fbins/1000, pnoise/gamma, label='proton')
plt.plot(fbins/1000, shot/gamma, label='shot')
plt.xscale('log')
plt.yscale('log')
plt.xlim([4, 256])
plt.ylim([1e-16, 5e-13])
plt.plot(fbins/1000, outliers)
plt.xlabel(r'$f[kHz]$')
plt.ylabel(r'$V_r^2[V^2Hz^{-1}]$')
plt.title('June 25 1995:06:33:06.432')
plt.legend(loc='best')
plt.show()

In [ ]:
a = list(range(100))
a_n =  np.random.normal(a, np.mean(a)/20)
plt.plot(a)
plt.plot(a_n)